In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from scipy.stats import randint
from scipy.io import loadmat
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)

## Bias Variance Tradeoff

This notebook will walk through one of the most fundamental concepts in machine learning, the bias variance tradeoff, by letting you experiment with various settings of hyperparameters for decision trees and SVMs. 

Note: In this homework, you will be seeing types of ML models (eg: decision trees) that we haven't covered in lecture. We hope that through this homework, you will see how each model has various hyper parameters that can be tuned to determine how much the model fits to the training data set, rather than focusing on how exactly the models work. Here are lecture slides from previous offerings of the class that go over decision trees and SVMs in more detail if you are interested:
    * Decision Trees: https://github.com/mlberkeley/Machine-Learning-Decal-Spring-2018/blob/master/Lecture%205/lecture5.pdf
    * SVMS: https://github.com/mlberkeley/Machine-Learning-Decal-Spring-2018/blob/master/Lecture%206/Lecture%206%20Slides.pdf
    

### Overview of Decision Trees

A decision tree is a model with a tree structure, that has a collection of if, then statements in its internal nodes. A new data point is routed through the tree based on the answer to the if, then statements until it reaches a leaf node, which represents the class. 

<img src="decision_tree.png">

To learn the features and values to split on, we greedily find the feature and value that best splits the data in the parent, so the child nodes can have more homogenous class values. This is formally known as minimizing entropy.

If left unconstrained, a decision tree can form as many layers as required to perfectly memorize the training dataset (eg: have exactly one training example in each leaf node). This results in a high variance model. To improve the generalization ability of decision trees, we can use various heuristics to limit the expressive capacity of the tree. Some examples are:
* Early stopping - limit the depth of the tree, or the threshold the purity of the classes of the leaves; limiting the depth of the tree, requiring a minimum number of samples per leaf, and setting the max number of nodes per leaf all prevent the tree from growing fully and reduce the variance
* Pruning - create the full tree and greedily remove trees to improve validation accuracy

We will now play around with various hyperparameters of decision trees using the Iris Dataset.

First load the iris dataset into a pandas dataframe.

In [ ]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

We will then add the species label to each datapoint using the encoding given in the dataset to understand what the overall dataset looks like.

In [ ]:
df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df.head()

Split the dataframe into training and validation data.

In [ ]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
train = train.drop(['is_train'], axis = 1)
test = test.drop(['is_train'], axis = 1)

Separate the labels and feature from both the training and test datasets and refactorize the labels

In [ ]:
train_features = train[train.columns[:4]]
train_labels = pd.factorize(train['species'])[0]
test_features = test[test.columns[:4]]
test_labels = pd.factorize(test['species'])[0]

Create a basic decision tree which minimizes entropy and fit it to the training data.

In [ ]:
clf = tree.DecisionTreeClassifier(criterion = 'entropy')
clf.fit(train_features, train_labels)

Compute the training and validation accuracies. You should notice the decision tree gets a higher training accuracy than validation. Decision trees have the capability to fit each training point accurately, so if left unregulated the tree will probably end up overfitting to the training set.

In [ ]:
clf.score(train_features, train_labels)

In [ ]:
clf.score(test_features, test_labels)

We will now use Grid Search to find a good set of hyperparameters which attempt to regualize the tree. Grid search allows us to pick the combination of hyperparameters that results in the best model. Feel free to play around with the parameters of grid search to find a better tree.

In [ ]:
parameters = {"min_samples_split": [2, 10],
              "max_depth": [None, 2, 5, 10],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20],
              }
gridsearch = GridSearchCV(clf, parameters)
gridsearch.fit(train_features, train_labels)

In [ ]:
best_tree = gridsearch.best_estimator_ 
best_tree.fit(train_features, train_labels)

TODO: Report the values of the hyperparameters that produced the best tree. Where do the final hyperparameters fall in the range that you tested? Why is this so? What is the tradeoff between regularizing the model and increasing its training accuracy?

Your answer here:

Compute the training and validation accuracies. On a dataset like Iris, the unregularized decision tree may already achieve the highest validation accuracy, but this method of regularization is useful when using decision trees for more complex tasks.

In [ ]:
best_tree.score(train_features, train_labels)

In [ ]:
best_tree.score(test_features, test_labels)

### Support Vector Machines
We will now use the SVM implementation in sklearn to classify 2D datasets and understand how to tune the hyperparameters of SVMS. This part of the notebook is adapted from http://nbviewer.jupyter.org/github/jdwittenauer/ipython-notebooks/blob/master/notebooks/ml/ML-Exercise6.ipynb

##### How to Choose C
Large C:
* Goal is to misclassify few training points
* Often results in small margins
* Very sensitive to outliers
* Risk of overfitting

Small C:
* Maximizes margin at cost of misclassifying training data points
* Risk of underfitting

We will now look at a nonlinear dataset:

In [ ]:
raw_data = loadmat('ex6data2.mat')

data = pd.DataFrame(raw_data['X'], columns=['X1', 'X2'])
data['y'] = raw_data['y']

positive = data[data['y'].isin([1])]
negative = data[data['y'].isin([0])]

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(positive['X1'], positive['X2'], s=30, marker='x', label='Positive')
ax.scatter(negative['X1'], negative['X2'], s=30, marker='o', label='Negative')
ax.legend()

For this data set we'll build a support vector machine classifier using the built-in RBF kernel and examine its accuracy on the training data. To visualize the decision boundary, this time we'll shade the points based on the predicted probability that the instance has a negative class label. We'll see from the result that it gets most of them right.

In [ ]:
from sklearn import svm
svc = svm.SVC(C=100, gamma=10, probability=True)
svc

In [ ]:
svc.fit(data[['X1', 'X2']], data['y'])
svc.score(data[['X1', 'X2']], data['y'])

In [ ]:
data['Probability'] = svc.predict_proba(data[['X1', 'X2']])[:,0]

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(data['X1'], data['X2'], s=30, c=data['Probability'], cmap='Reds')

TODO: Fill in the following function to use grid search to find the optimal value of C and gamma (hyperparameter of the RBF kernel):

In [ ]:
from sklearn import svm, grid_search
def svc_param_selection(X, y, nfolds):
    # Fill in the dots
    Cs = [...]
    gammas = [...]
    param_grid = {'C': Cs, 'gamma' : gammas}
    search = grid_search.GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    search.fit(X, y)
    search.best_params_
    return search.best_params_

In [ ]:
svc_param_selection(data[['X1', 'X2']], data['y'], 5)

TODO: Based on how you chose potential values to search over, how do you know you found a good value of C and gamma?

Your answer here:

TODO: Use your optimal values of C and gamma to train the SVM:

In [ ]:
# Fill in your optimal values of C and gamma; You should achieve a higher accuracy than before
svc = svm.SVC(C=..., gamma=..., probability=True)
svc

In [ ]:
svc.fit(data[['X1', 'X2']], data['y'])
svc.score(data[['X1', 'X2']], data['y'])

In [ ]:
data['Probability'] = svc.predict_proba(data[['X1', 'X2']])[:,0]

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(data['X1'], data['X2'], s=30, c=data['Probability'], cmap='Reds')

TODO: For a fixed value of C, modify the value of gamma and see how the model predicts on the training set. Do higher or lower values of gamma correspond to a high variance model?

For more information of the RBF kernel and its hyperparameters, you can look at:
* Slides 33-34 of https://github.com/mlberkeley/Machine-Learning-Decal-Spring-2018/blob/master/Lecture%206/Lecture%206%20Slides.pdf


In [ ]:
svc = svm.SVC(C=10, gamma=..., probability=True)
svc

In [ ]:
svc.fit(data[['X1', 'X2']], data['y'])
svc.score(data[['X1', 'X2']], data['y'])

In [ ]:
data['Probability'] = svc.predict_proba(data[['X1', 'X2']])[:,0]

fig, ax = plt.subplots(figsize=(12,8))
ax.scatter(data['X1'], data['X2'], s=30, c=data['Probability'], cmap='Reds')

Your answer here: